In [3]:
import numpy as np

from qpy import *

# from qpy.linalg import *
# from qpy.gates import *
# from qpy.measurement import *


In [4]:
sample = generate_ghz_state(3)
measured, output = measure(sample, 2)

print("Sample:", sample)
print("Measured:", measured)

Sample: [0.70710678 0.         0.         0.         0.         0.
 0.         0.70710678]
Measured: [0. 0. 0. 0. 0. 0. 0. 1.]


In [5]:
#toffoli gate 
def TOFFOLI(control1, control2, target, num_qubits):
    
    if control1 == target or control2 == target:
        raise ValueError("Control and Target qubits must be different.")
    
    if not (0<= control1 <= num_qubits) or not (0<= target <= num_qubits) or not (0<= control2 <= num_qubits):
        raise ValueError("Control and target qubits must be within range")
        
    # Ensure control_qubits are ordered in ascending order of indices
    c1, c2 = sorted([control1, control2])

    # Apply the controlled NOT gate with padding
    if c2 < target:   # If both controls are before the target
        operator = apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) 
        + apply_kron([*[I]*(c2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) 
        #+ apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(c2 - c1 - 1), np.outer(one,one), *[I]*(num_qubits - c2 - 1)]) or apply_kron([*[I]*(c1), np.outer(one,one), *[I]*(c2 - c1 - 1), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)])
        + apply_kron([*[I]*(c1), np.outer(one,one), *[I]*(c2 - c1 - 1), np.outer(one,one), *[I]*(target - c1 - 1), X, *[I]*(num_qubits - target - 1)])
    
    
    elif c1 < target < c2: # If target qubit is in the middle of both controls
        operator = apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) 
        + apply_kron([*[I]*(c2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) 
        #+ apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(c2 - c1 - 1), np.outer(one,one), *[I]*(num_qubits - c2 - 1)]) or apply_kron([*[I]*(c1), np.outer(one,one), *[I]*(c2 - c1 - 1), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)])
        + apply_kron([*[I]*(c1), np.outer(one,one), *[I]*(target - c1 - 1), X, *[I]*(c2 - target - 1), np.outer(one,one), *[I]*(num_qubits - c2 - 1)])
        
    elif target < c1: # if target is before both controls
        operator = apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(num_qubits - c1 - 1)]) 
        + apply_kron([*[I]*(c2), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)]) 
        #+ apply_kron([*[I]*(c1), np.outer(zero,zero), *[I]*(c2 - c1 - 1), np.outer(one,one), *[I]*(num_qubits - c2 - 1)]) or apply_kron([*[I]*(c1), np.outer(one,one), *[I]*(c2 - c1 - 1), np.outer(zero,zero), *[I]*(num_qubits - c2 - 1)])
        + apply_kron([*[I]*(target), X, *[I]*(c1 - target - 1), np.outer(one,one), *[I]*(c2 - c1 - 1), np.outer(one,one), *[I]*(num_qubits - c2 - 1)])
        
    return operator


In [11]:
tof = TOFFOLI(0,1,2,3)
print(tof)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [19]:
%debug

> c:\users\raisa\appdata\local\temp\ipykernel_38920\3628880613.py(17)TOFFOLI()

1
2
2
3
3
*** IndentationError: expected an indented block after 'if' statement on line 1
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [ ]:
def bell_pair():
    bell_state = np.kron(zero,zero)
    bell_state = np.dot(np.kron(H, I), bell_state)
    bell_state = np.dot(cnot, bell_state)
    
    return bell_state

print(bell_pair())

def teleport_quantum_state(original_state):
    """Teleports a quantum state using an entangled Bell state."""
    #num_qubits = len(original_state)
    bell_state = bell_pair()

    state = np.kron(original_state, bell_state)
    
    # Apply CNOT controlled by original state
    state = cnot_nonadj(0, 1, 3)@state
    # print(state)

    # Apply Hadamard to controlled qubit
    state = add_gate(H, 0, 3)@state
    # print(state)

    # Measure both control qubits
    state, outcome1 = measure(state, 0)
    # print(state)
    
    state, outcome2 = measure(state, 2)
    # print(state)

    # Apply classical communication and corrections
    classical_bits = str(outcome2) + str(outcome1)
    correction_map = {
        "00": np.eye(2),
        "01": X,
        "10": Z,
        "11": Y
    }

    correction = add_gate(correction_map[classical_bits], 2, 3)
    state = np.dot(correction, state)
    # print(state)

    return state

[0.70710678 0.         0.         0.70710678]


In [ ]:
stt = teleport_quantum_state([0,0])
print(stt)

AssertionError: 

In [ ]:
#%debug

> c:\users\raisa\onedrive\documents\qpy\qpy\measurement.py(22)measure()
     20     # P1 = np.real(np.trace(M1 @ state))
     21 
---> 22     assert np.isclose(P0 + P1, 1)
     23 
     24     outcome = 0 if np.random.rand() < P0 else 1



*** AssertionError
0.0
